In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import glob
import os
import numpy as np
import re
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2013436985499965351
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1723124942
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6897702803977209300
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

## 데이터 불러오기

In [4]:
txt_file_path = 'C:/users/juwon/practice_aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
txt_list[:5]

['C:/users/juwon/practice_aiffel/lyricist/data/lyrics\\adele.txt',
 'C:/users/juwon/practice_aiffel/lyricist/data/lyrics\\al-green.txt',
 'C:/users/juwon/practice_aiffel/lyricist/data/lyrics\\alicia-keys.txt',
 'C:/users/juwon/practice_aiffel/lyricist/data/lyrics\\amy-winehouse.txt',
 'C:/users/juwon/practice_aiffel/lyricist/data/lyrics\\beatles.txt']

In [5]:
raw_corpus = []
for txt_file in txt_list :
    with open(txt_file,'r',encoding='UTF-8') as f :
        sentences = f.read().splitlines()
        raw_corpus.extend(sentences)
print("데이터 크기:", len(raw_corpus))
print(raw_corpus[:10])

데이터 크기: 187088
['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


## 데이터 정제

In [6]:
def preprocessing_sentence(sentence) :
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r'([?.!,¿])', r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r'[^a-zA-Z?.,!¿]+', " ", sentence) #4
    sentence = sentence.strip() #5
    sentence = '<start> '+sentence+' <end>' #6
    return sentence

In [8]:
corpus = []
for sentence in raw_corpus :
    if len(sentence) == 0 : continue
    if sentence[-1] == ':' : continue
      
    preprocessed_sentence = preprocessing_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print(corpus[:5])

['<start> and i just want to make love to you <end>', '<start> love to you , ooh yeah <end>', '<start> love to you , yeah <end>', '<start> oh i just want to make love to you happy new year ! i heard that you re settled down <end>', '<start> that you found a girl and you re married now <end>', '<start> i heard that your dreams came true <end>', '<start> guess she gave you things i didn t give to you old friend , why are you so shy ? <end>', '<start> ain t like you to hold back or hide from the light i hate to turn up out of the blue , uninvited <end>', '<start> but i couldn t stay away , i couldn t fight it <end>', '<start> i had hoped you d see my face and that you d be reminded <end>']


In [10]:
def tokenize(corpus) : 
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000,
                                                     filters = '',
                                                     oov_token = "<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [sentence for sentence in tensor if len(sentence) <= 20] # 단어의 개수가 20개 이하인 문장 삭제
#     for i,j in enumerate(tensor) :
#         if len(j) > 20:
#             print(i,j)
#             del tensor[i]

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=20)
    return tensor, tokenizer

In [11]:
tensor, tokenizer = tokenize(corpus)
len(tensor)

169553

In [12]:
for i in tensor :
    if i[0] != 2 : 
        print(i)
    if i[-1] != 0 and i[-1] !=3 :
        print(i)  # <start> =2, <end> =3,<PAD> = 0 이므로 2로 시작, 0과3으로 끝나지 않는 문장 확인

In [13]:
for idx in tokenizer.index_word :
    print(idx, ":",tokenizer.index_word[idx])
    if idx >= 10 : break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [14]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]
print(src_input[0])
print(tgt_input[0])
len(src_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


19

## 데이터 셋 분리

In [15]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size = 0.2,
                                                         random_state=42)

## 모델 생성 및 학습

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 128 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.

model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [19]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none' # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)

model.compile(loss=loss, optimizer=optimizer)

with tf.device('/gpu:0'):
    history = model.fit(enc_train, dec_train,validation_data=(enc_val, dec_val),epochs= 10,batch_size= 256)

Epoch 1/10
530/530 [==============================] - 116s 196ms/step - loss: 2.8246 - val_loss: 2.5171
Epoch 2/10
530/530 [==============================] - 107s 201ms/step - loss: 2.4280 - val_loss: 2.3712
Epoch 3/10
530/530 [==============================] - 109s 206ms/step - loss: 2.3052 - val_loss: 2.2848
Epoch 4/10
530/530 [==============================] - 111s 209ms/step - loss: 2.2138 - val_loss: 2.2266
Epoch 5/10
530/530 [==============================] - 113s 213ms/step - loss: 2.1384 - val_loss: 2.1788
Epoch 6/10
530/530 [==============================] - 111s 210ms/step - loss: 2.0704 - val_loss: 2.1372
Epoch 7/10
530/530 [==============================] - 114s 216ms/step - loss: 2.0067 - val_loss: 2.1034
Epoch 8/10
530/530 [==============================] - 115s 216ms/step - loss: 1.9474 - val_loss: 2.0753
Epoch 9/10
530/530 [==============================] - 113s 213ms/step - loss: 1.8912 - val_loss: 2.0508
Epoch 10/10
530/530 [==============================] - 116s 219m

## 모델이 만든 가사 확인해보기 

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():큰
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna be <end> '

## 마치며

여태했던 Exploration 중에서 가장 힘들었다. 
1. 학습시간이 길다.
    * 처음시도 했을때 1에폭에 20-25분이었다. 10에폭이면 3-4시간정도 걸린다. 돌리지 않고 다른 방법을 찾기로 했다.
    * embedding과 hiddensize를 낮췄다. 1에폭에 10분정도 걸렸다. 하지만 loss가 4.xx로 나왔다.
    * GPU를 사용하면 학습이 빠르다고 해서 실행해보기로 했다. 여기서 시간을 엄청 까먹었다. 드라이버와 패키지 버전문제 때문에 삭제하고 다시 깔고를 반복했다.
    * GPU를 사용했을 때 1에폭에 2분 걸렸다. 효과는 굉장했다. 10배나 빨라졌다.
    * GPU를 사용했을때 GPU의 온도가 75도 까지 올라가서 처음에 무서웠지만 찾아보니까 70~80도가 정상이라고 해서 안심했다.
    * 학습을 편하게 하기 위해 GPU에 관해 더 찾아보아야겠다.
2. 단어 개수 제한
    * pad_sequences의 maxlen을 20으로 설정해 단어개수를 제한하면 앞이나 뒤에서부터 자르기때문에 단어가 20개가 넘는 문장의 경우시작,종료 토큰이 사라지게 되므로 학습에 영향이 있을 수 있다고 판단.
    * 따라서 패딩에 들어가기전에 단어의 개수가 20개인 문장만 남겨두기 위해 for와 enumerate를 사용했지만 지워지지 않는 문장도 있어서 '절대 그럴 수 없는데 왜 그럴까?' 이생각만 하다가 시간을 엄청보냈다.
    * 이유는 20개가 넘는 문장의 인덱스를 하나씩 지워주는데 하나씩 지우면서 문장들 인덱스가 갱신되어서(하나씩 땡겨짐) 20개넘는 문장들의 인덱스가 붙어있는 경우에는 지워지지 않았다.
    * 이유를 찾고나니 파이썬공부가 많이 부족하다고 느꼈다.